In [ ]:
# Set arguments
PrimaryKeys = ""
IsIncremental = False

SourceWorkspace= ""
SourceLakehouse = ""
SourceLakehouseName = ""
source_file_path = ""
source_file_name = ""
source_file_type = ""

TargetWorkspace = ""
TargetLakehouse = ""
TargetLakehouseName = ""
target_schema = ""
target_name = ""


# # CSV
CompressionType = 'infer'
ColumnDelimiter = ','
RowDelimiter = '\n'
EscapeCharacter = '"'
Encoding = 'UTF-8'
first_row_is_header = True
infer_schema = True


## Load Libraries

In [ ]:
import re
import datetime
import json
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import *
from notebookutils import mssparkutils
import uuid

## Define Starttime

In [ ]:
start_audit_time = datetime.datetime.now()

## Set Configuration

In [ ]:
#Make sure you have enabled V-Order

spark.conf.set("sprk.sql.parquet.vorder.enabled", "true")

## Set your loading paths

In [ ]:
#Set SourceFile and target Location
source_changes_data_path = f"abfss://{SourceWorkspace}@onelake.dfs.fabric.microsoft.com/{SourceLakehouse}/Files/{source_file_path}/{source_file_name}"
print(source_changes_data_path)

#Beware 
target_data_path = f"abfss://{TargetWorkspace}@onelake.dfs.fabric.microsoft.com/{TargetLakehouse}/Tables/{target_schema}{target_name}"
print(target_data_path)


## Load new from Data Landingzone

In [ ]:
#Read all incoming changes in Parquet format
dfDataChanged= spark.read\
                .format(source_file_type) \
                .option("header","true") \
                .load(f"{source_changes_data_path}")


In [ ]:
# Replace spaces with underscores in column names
new_columns = [col.replace(' ', '') for col in dfDataChanged.columns]

# Rename the columns
dfDataChanged = dfDataChanged.toDF(*new_columns)

## DQ Checks

In [ ]:
#split PKcolumns string on , ; or :
PrimaryKeys = str(PrimaryKeys)

PrimaryKeys = re.split('[, ; :]', PrimaryKeys)
#remove potential whitespaces around Pk columns
PrimaryKeys = [column.strip() for column in PrimaryKeys if column != ""]

key_columns = PrimaryKeys
print(f": {', '.join(key_columns)}")
# Check if all PK's exist in source
for pk_column in key_columns:
    if pk_column not in dfDataChanged.columns:
        raise ValueError(f"PK: {pk_column} doesn't exist in the source.")
        # Define all the Non-Key columns => HashExcludeColumns

read_key_columns = [column for column in dfDataChanged.columns if column in key_columns]

# Add a column with the calculated hash, easier in later stage of with multiple PK
dfDataChanged = (dfDataChanged
                .withColumn("HashedPKColumn", sha2(concat_ws("||", *read_key_columns), 256)))


## Check for Duplicates

In [ ]:
if dfDataChanged.select('HashedPKColumn').distinct().count() != dfDataChanged.select('HashedPKColumn').count():
    raise ValueError(f'Source file contains duplicated rows for PK: {", ".join(key_columns)}')

## Add Hash

In [ ]:
non_key_columns = [column for column in dfDataChanged.columns if column not in key_columns]

#add a hashed cloumn to detect changes
dfDataChanged = (dfDataChanged
                .withColumn("HashedNonKeyColumns", md5(concat_ws("||", *non_key_columns))))

#Add RecordLoadDate to see when the record arrived
dfDataChanged = dfDataChanged.withColumn('RecordLoadDate', current_timestamp())


## Read Original if exists

In [ ]:
#Check if Target exist, if exists read the original data if not create table and exit
if DeltaTable.isDeltaTable(spark, target_data_path):
    # Read original/current data
    dfDataOriginal = (spark
                        .read.format("delta")
                        .load(target_data_path)
                        )

else:
    # Use first load when no data exists yet and then exit 
    dfDataChanged.write.format("delta").mode("overwrite").save(target_data_path)
    TotalRuntime = str((datetime.datetime.now() - start_audit_time)) 

    # Your data
    result_data = {
        "CopyOutput":{
            "Total Runtime": TotalRuntime,
            "TargetSchema": target_schema,
            "TargetName" : target_name
        }
        }


    mssparkutils.notebook.exit(result_data)

## Merge table

In [ ]:
#merge table 
deltaTable = DeltaTable.forPath(spark, f'{target_data_path}')
if IsIncremental in [False, 'false', 'False']:
    print(' - Incremental Loading is not enabled, deletes are allowed')
    merge = deltaTable.alias('original') \
        .merge(dfDataChanged.alias('updates'), 'original.HashedPKColumn == updates.HashedPKColumn') \
        .whenNotMatchedInsertAll() \
        .whenMatchedUpdateAll('original.HashedNonKeyColumns != updates.HashedNonKeyColumns') \
        .whenNotMatchedBySourceDelete() \
        .execute()
elif IsIncremental not in [False, 'false', 'False']:
    print(' - Incremental Loading is enabled, deletes are not allowed')
    merge = deltaTable.alias('original') \
        .merge(dfDataChanged.alias('updates'), 'original.HashedPKColumn == updates.HashedPKColumn') \
        .whenNotMatchedInsertAll() \
        .whenMatchedUpdateAll('original.HashedNonKeyColumns != updates.HashedNonKeyColumns') \
        .execute()

## Exit notebook

In [ ]:
TotalRuntime = str((datetime.datetime.now() - start_audit_time)) 

# Your data
result_data = {
    "CopyOutput":{
        "Total Runtime": TotalRuntime,
        "TargetSchema": target_schema,
        "TargetName" : target_name
    }
    }


mssparkutils.notebook.exit(result_data)